In [1]:
import pandas as pd

In [2]:
# using the query in get_listings.sql to generate data
df = pd.read_csv('data/crmls_listing_history_2023_08_21.csv')
df.head()

,source_id,close_date,expiration_date,listing_contract_date,purchase_contract_date,withdrawn_date,list_price,current_price,original_list_price,close_price,price_change_timestamp,status_change_timestamp,status,previous_list_price,previous_status
0,371701067,NaN,18/01/24,18/07/23,NaN,NaN,1875000.0,1875000.0,1875000.0,NaN,NaN,2023-07-20 00:30:57,A,NaN,C
1,371701069,NaN,17/01/24,19/07/23,NaN,NaN,26000.0,26000.0,26000.0,NaN,NaN,2023-07-19 07:28:59,A,NaN,NaN
2,371701838,NaN,03/10/23,19/07/23,28/07/23,NaN,1200000.0,1200000.0,1200000.0,NaN,NaN,2023-08-13 17:00:00,P,NaN,U
3,371701840,14/08/23,19/01/24,19/07/23,23/07/23,NaN,999000.0,1100000.0,999000.0,1100000.0,NaN,2023-08-14 03:17:21,S,NaN,P
4,371701071,NaN,31/12/23,19/07/23,NaN,NaN,2248000.0,2248000.0,2248000.0,NaN,NaN,2023-07-12 11:12:33,A,NaN,NaN


In [3]:
df[df['status'] == 'X']

,source_id,close_date,expiration_date,listing_contract_date,purchase_contract_date,withdrawn_date,list_price,current_price,original_list_price,close_price,price_change_timestamp,status_change_timestamp,status,previous_list_price,previous_status
8,371701846,NaN,15/05/07,15/11/06,NaN,NaN,969000.0,969000.0,969000.0,NaN,NaN,2007-05-15 18:14:57,X,NaN,NaN
38,371701917,NaN,19/08/23,19/07/23,NaN,NaN,1950000.0,1950000.0,1950000.0,NaN,NaN,2023-08-20 01:00:46,X,NaN,A
76,371729243,NaN,04/08/23,26/07/23,NaN,31/07/23,2100.0,2100.0,2100.0,NaN,NaN,2023-08-05 01:00:05,X,NaN,W
98,371731323,NaN,19/12/06,19/10/06,NaN,NaN,1200000.0,1200000.0,12000000.0,NaN,NaN,2006-12-19 17:14:20,X,NaN,NaN
106,371731326,NaN,27/04/02,27/09/01,NaN,NaN,350000.0,350000.0,350000.0,NaN,NaN,NaN,X,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272843,369828904,NaN,20/08/23,20/08/22,NaN,07/09/22,1129999.0,1129999.0,1129999.0,NaN,2022-08-31 17:00:00,2023-08-20 17:00:00,X,1129999.0,W
272844,370983854,NaN,20/08/23,20/02/23,NaN,22/02/23,499000.0,499000.0,499000.0,NaN,2023-02-19 16:00:00,2023-08-20 17:00:00,X,499000.0,W
272845,371314224,NaN,NaN,26/04/23,NaN,NaN,808000.0,808000.0,808000.0,NaN,2023-04-25 17:00:00,2023-08-20 17:00:00,X,808000.0,NaN
272853,369974594,NaN,05/12/08,05/09/08,NaN,NaN,825000.0,825000.0,849000.0,NaN,NaN,2008-12-05 16:00:01,X,NaN,NaN


In [4]:
for c in df.columns:
    if '_date' in c:
        df[f'{c}'] = pd.to_datetime(df[f'{c}'], dayfirst=True, errors='coerce')
    elif '_price' in c:
        df[f'{c}'] = pd.to_numeric(df[f'{c}'], errors='coerce')
        df[f'{c}'] = [1 if not pd.isna(x) else 0 for x in df[f'{c}']]
    elif '_timestamp' in c:
        df[f'{c}'] = pd.to_datetime(df[f'{c}'], errors='coerce')

In [5]:
def flatten(row):
    tmp = {'source_id': [], 'state': [], 'time': []}
    for c in df.columns:
        if '_date' in c or '_timestamp' in c:
            tmp['source_id'].append(row['source_id'])
            tmp['state'].append(c)
            tmp['time'].append(row[c])
    return pd.DataFrame(tmp)

In [6]:
df_flat_list = [flatten(row) for index, row in df.iterrows()]

In [7]:
# df_flat = pd.concat(df_flat_list)
df_flat_list = [x.sort_values('time') for x in df_flat_list]

In [8]:
# df_flat.head()

In [9]:
# df_flat.shape

In [10]:
# df_flat.sort_values(by=['source_id', 'time'], inplace=True)

In [11]:
# df_flat

In [12]:
df_collapsed = {'source_id': [], 'key': []}
for d in df_flat_list:
    ret_val = ''
    source_id = None
    for i in d[~d['time'].isna()]['state']:
        ret_val += i
        ret_val += '|'
    source_id = d['source_id'].values[0]
    df_collapsed['source_id'].append(source_id)
    df_collapsed['key'].append(ret_val)
df_collapsed = pd.DataFrame(df_collapsed)
df_collapsed.head()

,source_id,key
0,371701067,listing_contract_date|status_change_timestamp|...
1,371701069,listing_contract_date|status_change_timestamp|...
2,371701838,listing_contract_date|purchase_contract_date|s...
3,371701840,listing_contract_date|purchase_contract_date|c...
4,371701071,status_change_timestamp|listing_contract_date|...


In [13]:
crmls_events_order = df_collapsed.groupby('key')['source_id'].nunique().sort_values(ascending=False)
print(crmls_events_order.head())
crmls_events_order.to_csv('data/crmls_events_order.csv')

key
listing_contract_date|status_change_timestamp|expiration_date|                                                             50565
listing_contract_date|purchase_contract_date|close_date|status_change_timestamp|expiration_date|                           44360
listing_contract_date|status_change_timestamp|price_change_timestamp|expiration_date|                                      17014
listing_contract_date|purchase_contract_date|status_change_timestamp|close_date|expiration_date|                           14555
listing_contract_date|price_change_timestamp|purchase_contract_date|close_date|status_change_timestamp|expiration_date|    12512
Name: source_id, dtype: int64


In [14]:
# df_flat.to_csv('data/crmls_events_sorted.csv')
df_collapsed.to_csv('data/crmls_events_collapsed.csv')

In [15]:
print(df.shape, df['source_id'].nunique())

(272920, 15) 272920


In [16]:
p_keys = []
for index, row in df.iterrows():
    price_keys = []
    for c in df.columns:
        if '_date' not in c and c != 'source_id' and 'status' not in c and '_timestamp' not in c:
            if not pd.isna(row[f'{c}']):
                price_keys.append(c)
    price_keys.sort()
    price_key = ''
    for p in price_keys:
        price_key += p
        price_key += '|'
    p_keys.append(price_key)
df['price_key'] = p_keys
df.head()

,source_id,close_date,expiration_date,listing_contract_date,purchase_contract_date,withdrawn_date,list_price,current_price,original_list_price,close_price,price_change_timestamp,status_change_timestamp,status,previous_list_price,previous_status,price_key
0,371701067,NaT,2024-01-18,2023-07-18,NaT,NaT,1,1,1,0,NaT,2023-07-20 00:30:57,A,0,C,close_price|current_price|list_price|original_...
1,371701069,NaT,2024-01-17,2023-07-19,NaT,NaT,1,1,1,0,NaT,2023-07-19 07:28:59,A,0,NaN,close_price|current_price|list_price|original_...
2,371701838,NaT,2023-10-03,2023-07-19,2023-07-28,NaT,1,1,1,0,NaT,2023-08-13 17:00:00,P,0,U,close_price|current_price|list_price|original_...
3,371701840,2023-08-14,2024-01-19,2023-07-19,2023-07-23,NaT,1,1,1,1,NaT,2023-08-14 03:17:21,S,0,P,close_price|current_price|list_price|original_...
4,371701071,NaT,2023-12-31,2023-07-19,NaT,NaT,1,1,1,0,NaT,2023-07-12 11:12:33,A,0,NaN,close_price|current_price|list_price|original_...


In [17]:
df_collapsed = df_collapsed.merge(df.drop(columns=[c for c in df.columns if ('date') in c or ('timestamp') in c]), how='left', on='source_id')
df_collapsed

,source_id,key,list_price,current_price,original_list_price,close_price,status,previous_list_price,previous_status,price_key
0,371701067,listing_contract_date|status_change_timestamp|...,1,1,1,0,A,0,C,close_price|current_price|list_price|original_...
1,371701069,listing_contract_date|status_change_timestamp|...,1,1,1,0,A,0,NaN,close_price|current_price|list_price|original_...
2,371701838,listing_contract_date|purchase_contract_date|s...,1,1,1,0,P,0,U,close_price|current_price|list_price|original_...
3,371701840,listing_contract_date|purchase_contract_date|c...,1,1,1,1,S,0,P,close_price|current_price|list_price|original_...
4,371701071,status_change_timestamp|listing_contract_date|...,1,1,1,0,A,0,NaN,close_price|current_price|list_price|original_...
...,...,...,...,...,...,...,...,...,...,...
272915,371831265,price_change_timestamp|status_change_timestamp...,1,1,1,0,A,0,NaN,close_price|current_price|list_price|original_...
272916,371831266,price_change_timestamp|status_change_timestamp...,1,1,1,0,C,0,NaN,close_price|current_price|list_price|original_...
272917,371831267,price_change_timestamp|status_change_timestamp...,1,1,1,0,A,0,NaN,close_price|current_price|list_price|original_...
272918,371831290,listing_contract_date|status_change_timestamp|...,1,1,1,0,C,0,NaN,close_price|current_price|list_price|original_...


In [18]:
df_collapsed.to_csv('data/crmls_listing_history_out.csv')